비전트랜스포머(Vit)
  - 컴퓨터 비전문제를 처리하기위해서 트랜스포머 아키텍처를 사용하는 모델
  - Vit는 이미지를 고정크기의 패치로 나누고 각 패치를 선형변환을 통해서 임베딩 벡터로 변환해서 입력으로 사용해서 self attention 매커니즘을 통해 이미지의 특징을 학습

ViT의 주요 구성 요소
  1. 이미지패치를 분할
    - 이미지를 N x N 크기의 작은 패치로 나눕니다
    - ex) 224 x 224 이미지를 16 x 16 패치로나누면 14 x 14 * 196개의 패치가생성
  2. 패치 임베딩
    - 각 패치는 16 x 16 x 3 크기의 벡터로 표현, 이를 선형 변환하여 고정된 차원의 임베딩으로 변환
    - ex) 16 x 16 x3 = 768 임베딩 차원 D = 512
  3. 위치 임베딩
     - 트랜스포머는 순서정보가 없다, 패치의 순서를 나타내기 위해 위치 임베딩을 추가
  4. 트랜스포머 인코더
    - 트랜스포머 아키텍처를 사용하여 패치 임베딩 간의 관계를 학습
      - 멀티헤드 셀프 어텐션
      - 피드포워드 네트웍
  5. 분류 토근(Class Token)
    - 추가적인 [CLS] 토큰을 삽입하여 최종적으로 이 토큰을 통해 이미지 분류 결과를 출력          



In [1]:
import torch
from torch import nn
from torchvision.transforms import Compose,Resize,ToTensor

In [2]:
import torch
from torch import nn
from torchvision.transforms import Compose, Resize, ToTensor

class VisionTransformer(nn.Module):
    def __init__(self, image_size=224, patch_size=16, num_classes=1000, dim=768, depth=12, heads=12, mlp_dim=3072, dropout=0.1):
        super().__init__()
        assert image_size % patch_size == 0, "Image dimensions must be divisible by the patch size."
        self.num_patches = (image_size // patch_size) ** 2
        self.patch_dim = patch_size * patch_size * 3

        # Patch + Position Embeddings
        self.patch_to_embedding = nn.Linear(self.patch_dim, dim)
        self.positional_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(dropout)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        # Classifier head
        self.to_cls_token = nn.Identity()
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        batch_size = img.shape[0]
        patches = img.unfold(2, 16, 16).unfold(3, 16, 16)
        patches = patches.contiguous().view(batch_size, -1, self.patch_dim)
        tokens = self.patch_to_embedding(patches)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, tokens), dim=1)
        x += self.positional_embedding
        x = self.dropout(x)

        x = self.transformer_encoder(x)  # Use TransformerEncoder
        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)

# Example usage
transform = Compose([Resize((224, 224)), ToTensor()])
vit_model = VisionTransformer()
dummy_image = torch.randn(1, 3, 224, 224)  # Batch size 1, RGB Image
output = vit_model(dummy_image)
print(output.shape)  # Output logits for each class


torch.Size([1, 1000])


주요 단계
1. 데이터 준비
  - CIFAR-10 데이터를 로드하고 전처리합니다.
  - 데이터 증강을 사용하여 성능을 향상시킵니다.
2. 모델 초기화
  - Vision Transformer를 생성합니다.
3. 손실 함수 및 옵티마이저 정의
  - CrossEntropyLoss와 AdamW 옵티마이저를 사용합니다.
4. 학습 루프
  - 배치별로 데이터를 처리하고 손실을 계산하여 역전파를 수행합니다.
  - 학습 진행 상황을 출력합니다.

In [3]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ViT는 고정된 입력 크기를 요구합니다.
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# CIFAR-10 데이터셋 로드
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Vision Transformer 모델 정의 (이전 코드 사용)
class VisionTransformer(nn.Module):
    def __init__(self, image_size=224, patch_size=16, num_classes=10, dim=768, depth=12, heads=12, mlp_dim=3072, dropout=0.1):
        super().__init__()
        assert image_size % patch_size == 0, "Image dimensions must be divisible by the patch size."
        self.num_patches = (image_size // patch_size) ** 2
        self.patch_dim = patch_size * patch_size * 3

        self.patch_to_embedding = nn.Linear(self.patch_dim, dim)
        self.positional_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        self.to_cls_token = nn.Identity()
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        batch_size = img.shape[0]
        patches = img.unfold(2, 16, 16).unfold(3, 16, 16)
        patches = patches.contiguous().view(batch_size, -1, self.patch_dim)
        tokens = self.patch_to_embedding(patches)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, tokens), dim=1)
        x += self.positional_embedding
        x = self.dropout(x)

        x = self.transformer_encoder(x)
        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)

100%|██████████| 170M/170M [00:03<00:00, 49.1MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VisionTransformer(num_classes=10).to(device)

In [5]:
# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

In [6]:
# 학습 루프
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 2.3486
Epoch [2/10], Loss: 2.3216


KeyboardInterrupt: 

In [ ]:
# 테스트 루프
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")